In [1]:
import json
import os
import tqdm

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import lib_stylegan


im_size = 128
latent_size = 256 
channels = 24 # Should be at least 32 for good results
#Chosing the number of layer this way, means we start with 4x4
nb_layer = int(np.log2(im_size) - 1) 

path="/Data/leo/dogs-face-2015/*jpg"

per_worker_batch_size = 32


strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

model = lib_stylegan.style_gan.StyleGan(im_size=im_size, 
                                            latent_size=latent_size, 
                                            channels=channels,
                                            nb_layer=nb_layer)
                                            #global_batch_size=global_batch_size)
model.compile(run_eagerly=True)

dataset = lib_stylegan.dataset.train_dataset(path, 
                                             n_layers=nb_layer, 
                                             im_size=im_size, 
                                             batch_size=per_worker_batch_size,
                                             latent_size=latent_size
                                            )
print("Here")
for args in dataset.take(1):
    model.tf_train_step(*(args+(False, False)))
    
for args in tqdm.tqdm(dataset.take(100), total=100):
    model.tf_train_step(*(args+(False, False)))

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.AUTO
Number of train images found: 52597
Here


100%|██████████| 100/100 [01:09<00:00,  1.44it/s]
